In [19]:
import pandas as pd
from datetime import datetime

In [2]:
data = {"Name": ['Hubert', 'Ola', 'Jadzia', 'Przemek', 'Iga'],
        "Age": [30, 30, 21, 31, 18],
        "City": ['Bdg', 'Gdynia', 'Wwa', 'Ino', 'Poznan'],
        "Salary" : [5000, 5500, 6500, 4350, 5000]}
df = pd.DataFrame(data)

In [3]:
df

,Name,Age,City,Salary
0,Hubert,30,Bdg,5000
1,Ola,30,Gdynia,5500
2,Jadzia,21,Wwa,6500
3,Przemek,31,Ino,4350
4,Iga,18,Poznan,5000


In [4]:
#Dodaj nową kolumnę Bonus, która wynosi 10% pensji (Salary).
df['Bonus'] = df['Salary'].apply(lambda x: x*0.1)
df

,Name,Age,City,Salary,Bonus
0,Hubert,30,Bdg,5000,500.0
1,Ola,30,Gdynia,5500,550.0
2,Jadzia,21,Wwa,6500,650.0
3,Przemek,31,Ino,4350,435.0
4,Iga,18,Poznan,5000,500.0


In [6]:
#Usuń kolumnę City
df = df.drop('City', axis=1)
df

,Name,Age,Salary,Bonus
0,Hubert,30,5000,500.0
1,Ola,30,5500,550.0
2,Jadzia,21,6500,650.0
3,Przemek,31,4350,435.0
4,Iga,18,5000,500.0


In [9]:
#Posortuj DataFrame według kolumny Salary w kolejności malejącej
df_sorted = df.sort_values(by=['Salary'], ascending=False)
df_sorted

,Name,Age,Salary,Bonus
2,Jadzia,21,6500,650.0
1,Ola,30,5500,550.0
0,Hubert,30,5000,500.0
4,Iga,18,5000,500.0
3,Przemek,31,4350,435.0


In [10]:
#2. Indeksowanie i filtrowanie danych

In [12]:
sales_df = pd.read_csv('sales.csv')
sales_df.head()

,SaleID,Region,Amount,Date
0,1,South,773,2024-06-16
1,2,South,179,2024-03-10
2,3,North,681,2024-10-09
3,4,North,651,2024-07-26
4,5,South,627,2024-11-15


In [14]:
sales_df['Date'] = pd.to_datetime(sales_df['Date'])
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   SaleID  100 non-null    int64         
 1   Region  100 non-null    object        
 2   Amount  100 non-null    int64         
 3   Date    100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 3.3+ KB


In [18]:
#Wyświetl wiersze, w których Amount > 500 i region to "North".
df_1 = sales_df[(sales_df['Amount'] > 500) & (sales_df['Region'] == "North")]
df_1

,SaleID,Region,Amount,Date
2,3,North,681,2024-10-09
3,4,North,651,2024-07-26
9,10,North,960,2024-12-26
14,15,North,585,2024-09-01
19,20,North,811,2024-09-27
29,30,North,682,2024-01-20
38,39,North,822,2024-05-06
42,43,North,611,2024-04-16
45,46,North,945,2024-10-10
53,54,North,810,2024-03-20


In [29]:

#Wybierz tylko kolumny Region i Amount dla transakcji, w których Date przypada na luty 2024.
s_date = datetime(2024,2,1)
e_date = datetime(2024,2,28)

df_luty = sales_df[(sales_df['Date'] >= s_date) & (sales_df['Date'] <= e_date)]
df_region_amount = df_luty[['Region', 'Amount']].reset_index(drop=True)
df_region_amount


,Region,Amount
0,North,233
1,East,804
2,West,628
3,North,344
4,East,823
5,West,662
6,South,382
7,East,669
8,South,571
9,West,661


In [31]:
#3. Grupowanie danych

tr_by_region = sales_df.groupby("Region")['Amount'].count()

avg_amount_by_region = sales_df.groupby("Region")["Amount"].mean()

print("Liczba transakcji w kazdym regionie:")
print(tr_by_region)
print("Srednia wartosc amount dla kazdego regionu:")
print(avg_amount_by_region)

Liczba transakcji w kazdym regionie:
Region
East     30
North    19
South    26
West     25
Name: Amount, dtype: int64
Srednia wartosc amount dla kazdego regionu:
Region
East     625.033333
North    623.421053
South    481.961538
West     552.080000
Name: Amount, dtype: float64


In [53]:
#Dodaj kolumnę Amount_Percentage, która wskazuje procentowy udział transakcji danego regionu w całkowitej sprzedaży.
total_sales = sales_df['Amount'].sum()
print(total_sales)

total_amount_by_region = sales_df.groupby("Region")['Amount'].sum()

a = round((total_amount_by_region / total_sales) * 100,2)
a_df = a.reset_index()

#merging into one df
sales_df = sales_df.merge(a_df, how="left", on="Region")

sales_df.head()


56929


,SaleID,Region,Amount,Date,Amount_Percentage
0,1,South,773,2024-06-16,22.01
1,2,South,179,2024-03-10,22.01
2,3,North,681,2024-10-09,20.81
3,4,North,651,2024-07-26,20.81
4,5,South,627,2024-11-15,22.01


In [51]:
sales_df = sales_df.rename(columns={"Amount_x": "Amount", "Amount_y" : "Amount_Percentage"})

In [60]:
#Tworzenie i modyfikacja kolumn

def apply_discount(amount):
    if amount > 700:
        return amount * 0.15
    elif amount >= 500 and amount <= 700:
        return amount * 0.1
    elif amount < 500:
        return 0
    
sales_df['Discount'] = sales_df['Amount'].apply(apply_discount)

sales_df.head()

,SaleID,Region,Amount,Date,Amount_Percentage,Discount
0,1,South,773,2024-06-16,22.01,115.95
1,2,South,179,2024-03-10,22.01,0.00
2,3,North,681,2024-10-09,20.81,68.10
3,4,North,651,2024-07-26,20.81,65.10
4,5,South,627,2024-11-15,22.01,62.70


In [61]:
sales_df["Final_Amount"] = sales_df['Amount'] - sales_df['Discount']
sales_df.head()

,SaleID,Region,Amount,Date,Amount_Percentage,Discount,Final_Amount
0,1,South,773,2024-06-16,22.01,115.95,657.05
1,2,South,179,2024-03-10,22.01,0.00,179.00
2,3,North,681,2024-10-09,20.81,68.10,612.90
3,4,North,651,2024-07-26,20.81,65.10,585.90
4,5,South,627,2024-11-15,22.01,62.70,564.30


In [62]:
df_missing_data = pd.read_csv('sales_missing_data.csv')
df_missing_data.head()

,SaleID,Region,Amount,Date
0,1,South,773.0,2024-06-16
1,2,South,179.0,2024-03-10
2,3,North,681.0,2024-10-09
3,4,North,NaN,2024-07-26
4,5,South,627.0,2024-11-15


In [64]:
#Wyświetl wiersze, które zawierają brakujące wartości.
df_missing_data.isna().sum()
#result -> SaleID     0
#Region     2
#Amount    10
#Date       0

SaleID     0
Region     2
Amount    10
Date       0
dtype: int64

In [68]:
#Wyświetl wiersze, które zawierają brakujące wartości.
missing_data = df_missing_data[df_missing_data.isna().any(axis=1)].reset_index(drop=True)
missing_data


,SaleID,Region,Amount,Date
0,4,North,NaN,2024-07-26
1,11,East,NaN,2024-05-08
2,18,West,NaN,2024-10-05
3,21,East,NaN,2024-01-05
4,24,NaN,640.0,2024-03-03
5,32,NaN,550.0,2024-01-16
6,33,South,NaN,2024-06-06
7,52,East,NaN,2024-06-27
8,56,West,NaN,2024-11-09
9,61,South,NaN,2024-12-28


In [75]:
#Uzupełnij brakujące wartości w kolumnie Amount wartością średnią dla tej kolumny.
avg_amount = round(df_missing_data["Amount"].mean(),2)
print(avg_amount)
df_missing_data = df_missing_data.fillna(value=avg_amount)


576.04


In [77]:
df_missing_data['MissingCount'] = df_missing_data.isna().sum(axis=1)
df_missing_data['MissingCount'].tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [78]:
df_cleaned = df_missing_data[df_missing_data['MissingCount'] <= 1].drop(columns=['MissingCount'])
df_cleaned

,SaleID,Region,Amount,Date
0,1,South,773.00,2024-06-16
1,2,South,179.00,2024-03-10
2,3,North,681.00,2024-10-09
3,4,North,576.04,2024-07-26
4,5,South,627.00,2024-11-15
...,...,...,...,...
95,96,South,717.00,2024-05-21
96,97,West,561.00,2024-04-01
97,98,South,385.00,2024-01-23
98,99,East,744.00,2024-07-16
